# 패키지 불러오기

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb

from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


n_splits = 5

# 모델 정의
 1. Random Forest
 2. Lightgbm
 3. Xgboost
 4. Catboost
 5. Knn
 6. Linear regression

In [2]:
def Xy_split(dataset):
    
    X = dataset.drop('연봉', axis=1)
    column_dict = {f'variable{idx+1}':col for idx, col in enumerate(X.columns)}
    X = X.rename(columns = {col:f'variable{idx+1}' for idx, col in enumerate(X.columns)})
    y = dataset['연봉']
    
    return column_dict, X, y

In [3]:
def rf_model(data_x, data_y):

    # rf 모델링
    performance = []
    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        rf_model = RandomForestRegressor(random_state = 42)
        rf_model.fit(X_train, y_train)

        pred_valid = rf_model.predict(X_valid)
        rmse = mean_squared_error(y_valid, pred_valid, squared=False)
        performance.append(rmse)
        
    # rf 성능종합
    performance.append(np.mean(performance))

    output = pd.DataFrame({'rf':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [4]:
def lgbm_model(data_x, data_y):

    # lgbm 모델링
    lgb_params = {"objective" : "rmse",
                 "verbosity" : -1}

    lgb_models={}
    step = 1

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        print(f'\n\n ============================ {step} ============================')    

        X_train = data_x.iloc[tr_idx, :].values
        y_train = data_y[tr_idx].values

        X_valid = data_x.iloc[val_idx, :].values
        y_valid = data_y[val_idx].values

        lgb_dtrain = lgb.Dataset(data = X_train, label = y_train) 
        lgb_dvalid = lgb.Dataset(data = X_valid, label = y_valid) 

        lgb_model = lgb.train(lgb_params, lgb_dtrain, 20000, valid_sets=[lgb_dvalid], early_stopping_rounds=100, verbose_eval=1000)
        lgb_models[step] = lgb_model

        step += 1
        
    # lgbm 성능종합
    performance = [lgb_models[step].best_score['valid_0']['rmse'] for step in range(1,6)]
    performance.append(np.mean(performance))

    output = pd.DataFrame({'lgbm':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [5]:
def xgb_model(data_x, data_y):
    
    # xgb 모델링
    xgb_final_param = {
          "objective" : 'reg:squarederror',
          "random_state" : 42,
          "verbosity" : 0
          }

    xgb_models={}
    step = 1

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :].values
        y_train = data_y[tr_idx].values

        X_valid = data_x.iloc[val_idx, :].values
        y_valid = data_y[val_idx].values

        xgb_dtrain = xgb.DMatrix(data = X_train, label = y_train) 
        xgb_dvalid = xgb.DMatrix(data = X_valid, label = y_valid) 

        xgb_model = xgb.train(params = xgb_final_param, dtrain = xgb_dtrain, num_boost_round = 20000, early_stopping_rounds = 100, verbose_eval = 1000, evals=[(xgb_dtrain, 'train'), (xgb_dvalid,'eval')])
        xgb_models[step] = xgb_model

        step += 1
        
    # xgb 성능종합
    performance = [xgb_models[step].best_score for step in range(1,6)]
    performance.append(np.mean(performance))

    output = pd.DataFrame({'xgb':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [6]:
def cb_model(data_x, data_y):

    # cb 모델링
    cat_cols = []

    cb_models={}
    step = 1

    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        cb_dtrain = Pool(data=X_train, label=y_train, cat_features=cat_cols)
        cb_dvalid = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

        cb_model = CatBoostRegressor(iterations=20000, eval_metric='RMSE', loss_function='RMSE', verbose = 0)

        cb_model.fit(cb_dtrain, eval_set=cb_dvalid, early_stopping_rounds=100, verbose_eval=1000, use_best_model=True)
        cb_models[step] = cb_model

        step += 1
        
    # cb 성능종합
    performance = [cb_models[step].best_score_['validation']['RMSE'] for step in range(1,6)]
    performance.append(np.mean(performance))

    output = pd.DataFrame({'cb':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [7]:
def knn_model(data_x, data_y):

    # knn 모델링
    performance = []
    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        knn_model = KNeighborsRegressor()
        knn_model.fit(X_train, y_train)

        pred_valid = knn_model.predict(X_valid)
        rmse = mean_squared_error(y_valid, pred_valid, squared=False)
        performance.append(rmse)
        
    # knn 성능종합
    performance.append(np.mean(performance))

    output = pd.DataFrame({'knn':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

In [8]:
def regression_model(data_x, data_y):

    # regression 모델링
    performance = []
    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        regression_model = LinearRegression()
        regression_model.fit(X_train, y_train)

        pred_valid = regression_model.predict(X_valid)
        rmse = mean_squared_error(y_valid, pred_valid, squared=False)
        performance.append(rmse)
        
    # regression 성능종합
    performance.append(np.mean(performance))

    output = pd.DataFrame({'regression':performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

# train, test분할 후 모델링

In [9]:
hitter = pd.read_csv('../선수데이터(전처리완료)/모델링용ver2/타자(모델링용).csv')
pitcher = pd.read_csv('../선수데이터(전처리완료)/모델링용ver2/투수(모델링용).csv')

#### 1. 타자

In [10]:
# train, test 분할
col_dict, X, y = Xy_split(hitter)
X, X_test, y, y_test = train_test_split(X, y, test_size = 0.25, shuffle = True, random_state = 42)

X = X.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y = y.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [11]:
# 타자데이터 도출
hitter_performance = pd.concat([rf_model(X, y),
                                  lgbm_model(X, y),
                                  xgb_model(X, y),
                                  cb_model(X, y),
                                  knn_model(X, y),
                                  regression_model(X, y)], axis=1)



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[77]	valid_0's rmse: 20255.4


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[35]	valid_0's rmse: 17305.9


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	valid_0's rmse: 14824.5


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[80]	valid_0's rmse: 20533.1


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[27]	valid_0's rmse: 14675.6
[0]	train-rmse:29327.75391	eval-rmse:37808.40625
Multip

#### 2. 투수

In [12]:
# train, test 분할
col_dict, X, y = Xy_split(pitcher)
X, X_test, y, y_test = train_test_split(X, y, test_size = 0.25, shuffle = True, random_state = 42)

X = X.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y = y.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [13]:
# 타자데이터 도출
pitcher_performance = pd.concat([rf_model(X, y),
                                  lgbm_model(X, y),
                                  xgb_model(X, y),
                                  cb_model(X, y),
                                  knn_model(X, y),
                                  regression_model(X, y)], axis=1)



 ============================ 1 ============================
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's rmse: 13280
Early stopping, best iteration is:
[1672]	valid_0's rmse: 13273.8


 ============================ 2 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[148]	valid_0's rmse: 13513.7


 ============================ 3 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[707]	valid_0's rmse: 13081.8


 ============================ 4 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[461]	valid_0's rmse: 16650.6


 ============================ 5 ============================
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[202]	valid_0's rmse: 13322.3
[0]	train-rmse:22789

In [17]:
hitter_performance

,rf,lgbm,xgb,cb,knn,regression
cv1,20364.028346,20255.353866,21237.861328,19923.875678,27211.080599,24470.106906
cv2,17410.972400,17305.864862,17409.261719,16661.318951,22557.946304,20909.793004
cv3,16557.807454,14824.539679,15893.333008,14655.965492,23774.938820,20360.048397
cv4,21528.249315,20533.064038,23610.375000,20702.597548,24159.600039,22261.974938
cv5,16597.428939,14675.557807,16400.156250,14804.248754,20677.378642,19512.435766
평균,18491.697291,17518.876050,18910.197461,17349.601285,23676.188881,21502.871802


In [15]:
pitcher_performance

,rf,lgbm,xgb,cb,knn,regression
cv1,13693.639460,13273.763512,13436.646484,12816.071612,16797.845898,18030.731517
cv2,14422.518499,13513.670366,14553.916992,12610.440759,14747.925366,18683.344279
cv3,14586.515669,13081.766544,15061.731445,13012.010752,15773.040503,18764.417096
cv4,16150.821558,16650.574801,16791.222656,15051.184642,19612.308150,20301.821560
cv5,13829.670562,13322.343031,14843.157227,13220.945061,16409.400335,17257.196151
평균,14536.633150,13968.423651,14937.334961,13342.130565,16668.104051,18607.502121
